In [ ]:
from queryrunner_client import Client 

qr = Client(user_email='will.zou@uber.com') 


In [ ]:
def make_query_daily(yesterday_ds):
    return """
        WITH xp AS (
            SELECT user_uuid
            , first_segment_key AS segment
            , first_treatment_group_key AS test_group
            , begin_effective_timestamp
            , TO_DATE(FROM_UNIXTIME(begin_effective_timestamp)) AS begin_effective_datestr
            FROM driver.carbon_m1_seut
            WHERE experiment_key = 'carbon_m1'
            AND datestr >= '2018-04-02'
            AND first_treatment_group_key IN ('carbon','control')
            AND first_segment_key IN ('carbon_m1_driver_intl','carbon_m1_driver_usa','carbon_m1_courier_intl','carbon_m1_courier_usa')
            AND begin_effective_timestamp >= UNIX_TIMESTAMP('2018-04-02 00:00:00')
            AND begin_effective_timestamp <= UNIX_TIMESTAMP('{yesterday_ds} 23:59:59')
            AND COALESCE(end_effective_timestamp, UNIX_TIMESTAMP('{yesterday_ds} 23:59:59')) >= UNIX_TIMESTAMP('{yesterday_ds} 23:59:59')
        )

        SELECT *
        FROM
        (
            SELECT xp.segment AS segment
            , (CASE WHEN xp.test_group = 'control' THEN 'ALLOY' ELSE 'CARBON' END) AS test_group
            , uwt.span_name AS metric
            , uwt.duration_us/1000000.0 AS metric_value
            FROM xp
            JOIN driver.carbon_uwt_driver_uuid uwt
            ON xp.user_uuid = uwt.driver_uuid
            WHERE uwt.datestr = '{yesterday_ds}'
            AND uwt.span_name IN ('cold_start_completed_startup','go_online','go_offline','begin_trip','end_trip')
            AND (uwt.start_time_us/1000000.0) >= xp.begin_effective_timestamp

            UNION ALL

            SELECT
            (CASE WHEN xp.segment IN ('carbon_m1_driver_intl','carbon_m1_driver_usa') THEN 'carbon_m1_driver_combined'
                  WHEN xp.segment IN ('carbon_m1_courier_intl','carbon_m1_courier_usa') THEN 'carbon_m1_courier_combined'
                  END) AS segment
            , (CASE WHEN xp.test_group = 'control' THEN 'ALLOY' ELSE 'CARBON' END) AS test_group
            , uwt.span_name AS metric
            , uwt.duration_us/1000000.0 AS metric_value
            FROM xp
            JOIN driver.carbon_uwt_driver_uuid uwt
            ON xp.user_uuid = uwt.driver_uuid
            WHERE uwt.datestr = '{yesterday_ds}'
            AND uwt.span_name IN ('cold_start_completed_startup','go_online','go_offline','begin_trip','end_trip')
            AND (uwt.start_time_us/1000000.0) >= xp.begin_effective_timestamp

        ) t
        ORDER BY RAND()
        LIMIT 3000000
    """.format(yesterday_ds=yesterday_ds)

In [ ]:
command = 
execution = qr.execute('hive', command) 
df_raw = pd.DataFrame(execution.load_data()) 
